# Classification example demonstrating stacking and mean-encoding

In [48]:
!pip install catboost

     |████████████████████████████████| 67.3 MB 12.1 MB/s eta 0:00:01     |███████████████████████▌        | 49.4 MB 11.8 MB/s eta 0:00:02
     |████████████████████████████████| 13.2 MB 15.2 MB/s eta 0:00:01    |███████                         | 2.9 MB 15.2 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=864283ba0f4d4f941ef40c574cd71f76f01a1ecc9fea3178932d83dc67d5ff02
  Stored in directory: /home/martin/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [50]:
from catboost.datasets import amazon
employee_train, employee_test = amazon()

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [51]:
y = employee_train['ACTION']
X = employee_train.drop('ACTION', axis=1)

# Split into train & validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)

In [ ]:
model = CatBoostClassifier(custom_metric=['TotalF1'], early_stopping_rounds=100, eval_metric='AUC')

model.fit(X_train, y_train, cat_features=cat_features,
          eval_set=(X_val, y_val), plot=True, verbose=False, use_best_model=True)


In [2]:
employee =employee.astype('category')

In [3]:
train, test, train_label, test_label = train_test_split(employee, y, test_size=0.2, random_state=42, stratify=y)

# test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')

In [4]:
train.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
10137,75834,3253,117961,118446,120317,208126,133698,124487,208127
26537,105046,70062,117961,118386,118746,117905,117906,290919,117908
25420,81465,43,119920,119921,119922,118321,117906,290919,118322
8690,17249,4950,117961,118343,119598,120006,310997,118424,120008
12147,45300,59765,117961,118386,121961,124152,304629,118424,124154


In [5]:
display(train.shape, test.shape)

(26215, 9)

(6554, 9)

In [6]:
categorical = [var for var in train.columns if train[var].dtype=='category']
numerical = [var for var in train.columns if train[var].dtype!='category']

In [7]:
categorical

['RESOURCE',
 'MGR_ID',
 'ROLE_ROLLUP_1',
 'ROLE_ROLLUP_2',
 'ROLE_DEPTNAME',
 'ROLE_TITLE',
 'ROLE_FAMILY_DESC',
 'ROLE_FAMILY',
 'ROLE_CODE']

In [8]:
!pip install category_encoders

In [9]:
import sys
import os
import numpy as np
sys.path.append(os.path.abspath('../scripts'))
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from validatedstackedmeanencoder import ValidatedStackedMeanEncoder

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler())])
     

#preprocessor1 = ColumnTransformer(
#    transformers=[
#        ('num', numeric_transformer, numerical),
#        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical)
#        ])    
    
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        #('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical),
        ('mean_enc', ValidatedStackedMeanEncoder(), categorical)
        ])

In [44]:
import lightgbm
model = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.035, 
                                n_estimators=5000, subsample_for_bin=20000, objective='binary', 
                                subsample=1.0, subsample_freq=0, colsample_bytree=1.0, 
                                n_jobs=- 1, silent=True, importance_type='split',
                                is_unbalance = False, scale_pos_weight = 1.0, random_state=42, metrics='AUC',
                                verbose=100)

custom_pipeline = make_pipeline(
            #preprocessor1,
            model
            )
from sklearn.model_selection import StratifiedKFold
custom_pipeline.fit(train, train_label)
#custom_pipeline.predict(test)
print("model score: %.3f" % custom_pipeline.score(test, test_label))

[LightGBM] [Info] Number of positive: 24697, number of negative: 1518
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.009750
[LightGBM] [Debug] init for col-wise cost 0.000013 seconds, init for row-wise cost 0.001665 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 5773
[LightGBM] [Info] Number of data points in the train set: 26215, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.942094 -> initscore=2.789288
[LightGBM] [Info] Start training from score 2.789288
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 20
[LightGBM] [Debug] T

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 21
[LightGBM] [Debug] T

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] T

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] T

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 26 and max_depth = 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_depth = 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 25 and max_depth = 7
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 19 and max_depth = 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 27 and max_depth = 11
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_depth = 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 21 and max_depth = 7
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 23 and max_depth = 11
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 21 and max_depth = 7
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 28 and max_depth = 11
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 21 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 25 and max_

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 21 and max_depth = 7
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_depth = 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 23 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 27 and max_depth = 11
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 23 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 26 and max_d

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 24 and max_depth = 13
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 25 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 25 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 26 and max_depth = 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 23 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 19 and max_depth = 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 19 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 25 and max_depth = 11
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and max_

In [45]:
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
model_validation = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0)
#model_val = enumerate(model_validation.split(employee, y))
target_encoder = CatBoostEncoder

#i, (train_idx, test_idx) = next(model_val


In [46]:
custom_pipeline = make_pipeline(
            preprocessor2,
            model
            )
from sklearn.model_selection import StratifiedKFold
custom_pipeline.fit(train, train_label)
#custom_pipeline.predict(test)
print("model score: %.3f" % custom_pipeline.score(test, test_label))

in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
in fit_transform
[LightGBM] [Info] Number of positive: 24697, number of negative: 1518
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000031
[LightGBM] [Debug] init for col-wise cost 0.000011 seconds, init for row-wise cost 0.001000 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000508 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 4090
[LightGBM] [Info] Number of data points in the train set: 26215, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.942094 -> initscore=2.789288
[LightGBM] [Info] 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Tra

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Tr

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Tra

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trai

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trai

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trai

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 13
[LightGBM] [Debug] Trai

in else of transform
model score: 0.947


In [14]:
probe = preprocessor2.transform(test)

in else of transform


In [15]:
pd.DataFrame(probe).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,33111.0,26143.0,118212.0,119763.0,123173.0,124537.0,199592.0,118870.0,124539.0,0.997213,0.987968,0.956842,0.699405,0.958388,0.996870,0.988226,0.998951,0.996870
1,73110.0,46170.0,118290.0,118291.0,118889.0,118028.0,307233.0,117887.0,118030.0,0.804417,0.992065,0.891536,0.890855,0.998609,0.807608,0.997323,0.956414,0.807608
2,25985.0,2685.0,122880.0,122974.0,117945.0,117946.0,149467.0,292795.0,117948.0,0.871328,0.978122,0.938633,0.938633,0.864023,0.833809,0.987840,0.857900,0.833809
3,37639.0,13133.0,118106.0,118107.0,117884.0,117885.0,117913.0,117887.0,117888.0,0.996190,0.942094,0.929845,0.929845,0.923992,0.963175,0.964730,0.956414,0.963175
4,25993.0,5106.0,117961.0,118300.0,118597.0,307024.0,311622.0,118331.0,118332.0,0.950553,0.965259,0.949659,0.956728,0.999260,0.913937,0.878115,0.937154,0.913937


In [ ]:
preprocessor2.transformers[1][1].list_of_encoders

In [ ]:
preprocessor2.transformers[1][1].list_of_encoders.append(target_encoder)

In [ ]:
my_list = [1, 2, 3]
for i in my_list:
    print(i)

In [ ]:
my_list

In [ ]:
pd.DataFrame(probe).head()

In [ ]:
probe =pd.DataFrame(probe)
probe.shape

In [ ]:
import numpy as np
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

In [ ]:
target_encoder = CatBoostEncoder
k_folds = 5
n_repeats = 3
model_validation = RepeatedStratifiedKFold(5, 3, random_state=42)
list_of_encoders = []

In [ ]:
X = pd.DataFrame(train).reset_index(drop=True)
y = pd.Series(train_label).reset_index(drop=True)
cols_representation = np.zeros_like(X)
cols = X.columns
for n_fold, (train_idx, val_idx) in enumerate(model_validation.split(X, y)):

    encoder = target_encoder(X.columns)
    X_train, X_val = X.loc[train_idx, :], X.loc[val_idx, :]
    y_train, y_val = y[train_idx], y[val_idx]
    _ = encoder.fit_transform(X_train, y_train)

    # transform validation part and get all necessary cols
    val_t = encoder.transform(X_val)


    cols_representation[val_idx, :] += val_t / n_repeats
    list_of_encoders.append(encoder)

cols_representation = pd.DataFrame(cols_representation, columns=['mean_enc_' + c for c in cols])
X = pd.concat([X, cols_representation], axis=1)
X.drop(cols, axis=1, inplace=True)


In [ ]:
cols_representation[val_idx, : ] += val_t / 3

In [ ]:
cols_representation[val_idx, :]

In [ ]:
cols_representation = np.zeros_like(X, dtype=float)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(
            preprocessor,
            GridSearchCV(
            GradientBoostingClassifier(),
            #lightgbm.LGBMClassifier(),
            param_grid={'subsample': [0.9, 1.0],
                        'max_depth': [3, 5, 9],
                        'n_estimators': [400, 500, 600],
                        'learning_rate' : 10**(np.linspace(-2.5, -1, 4))},
         cv=5, refit=False, scoring = 'accuracy', n_jobs=-1
)
)

In [ ]:
clf.fit(train, train_label)

In [ ]:
clf[1].best_params_

In [ ]:
new_Pipe = make_pipeline(
            preprocessor,
            GradientBoostingClassifier(**clf[1].best_params_)
            #lightgbm.LGBMClassifier()
)
new_Pipe.fit(train, train_label)


In [ ]:
test.head()

In [ ]:
type(y)